# §6 状態空間モデル
~ State Space Model with Kalman filter

#### Packageの読み込み

In [13]:
import numpy as np
import pandas as pd
#import pandas_profiling
import pandas.tseries.offsets as offsets

import scipy as sp
from scipy import stats
from scipy import signal

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import mlab

import seaborn as sns

import lightgbm as lgb

import datetime as dt
import math
import itertools
import warnings
from tqdm import tqdm_notebook as tqdm

#### Dataの読み込み（セクション2のまとめ）

In [14]:
df1 = pd.read_csv("./csv/japan/newly_confirmed_cases_daily.csv")

# Indexを変更し日付型に変換
df2 = df1.set_index("Date")
df2.index = pd.to_datetime(df2.index)

# ①原系列の定義
df3 = df2[(df2.index >= '2020-05-01') & (df2.index <= '2023-04-30')]
df = df3[["ALL"]].copy()
df.rename(columns={"ALL": "original"}, inplace = True)

#### その他の変数の定義（セクション3のまとめ）
(セクション4のまとめなし)

In [15]:
# ②移動平均の定義
df["MA"] = df["original"].rolling(window=7,min_periods=1).mean()

# ③対数系列の定義
c = 1e-5#ゼロを含むため極小さな値を入れる
df["log"] = np.log(df["original"]+c)

# ④一次差分系列の定義
df['1day_diff'] = df["original"].diff(1)

# ⑤季節階差（週トレンド）の定義
df['7day_diff'] =df ["original"].diff(7)

# ⑥対数1次差分系列
df["log_1day_diff"] = df["log"].diff(1) 

# ⑦対数7次差分系列
df["log_7day_diff"] = df["log"].diff(7)

# 欠損値の確認
df.head()
print(df.isna().sum())

original         0
MA               0
log              0
1day_diff        1
7day_diff        7
log_1day_diff    1
log_7day_diff    7
dtype: int64


#### 特徴量エンジニアリング

In [16]:
#原系列に対するラグ特徴量
df['lag1'] = df['original'].shift(1)
df['lag7'] = df['original'].shift(7)

#原系列に対する前日からの変化率特徴量
df['change_rate'] = df['original'].diff(1) / df['original'].shift(1)

#対数系列に対するラグ特徴量
df['loglag1'] = df['log'].shift(1)
df['loglag7'] = df['log'].shift(7)

#対数系列に対する前日からの変化率特徴量
df['log_change_rate'] = df['log'].diff(1) / df['log'].shift(1)

# ラグ特徴量によりNaNが生成されるので除去
df.dropna(inplace=True) 

#### 学習データと検証データを分割

In [29]:
train_df = df["2022-01-01":"2022-03-14"]
test_df = df["2022-03-15":"2022-03-31"]

#### 説明変数と目的変数を分割

In [30]:
y_train = train_df["original"]
X_train = train_df.drop(columns=["original"])

y_test = test_df["original"]
X_test = test_df.drop(columns=["original"])

In [19]:
# モデルの定義
model = SARIMAX(train_df['original'],
                order=(1, 1, 1),  # ARIMAのパラメータ (p, d, q)
                seasonal_order=(1, 1, 1, 7),  # 季節成分のパラメータ (P, D, Q, s)
                # カルマンフィルタの設定
                kalman_filter='exact')#変えられる★

# モデルの学習
fitted_model = model.fit()

# モデルによる予測
pred = fitted_model.get_forecast(steps=len(test_df))
y_pred = pred.predicted_mean
y_test = test_df['original']

# 評価指標の計算
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.38790D+00    |proj g|=  1.03909D-01

At iterate    5    f=  9.36512D+00    |proj g|=  5.62828D-02

At iterate   10    f=  9.35478D+00    |proj g|=  1.42079D-03

At iterate   15    f=  9.35477D+00    |proj g|=  1.78472D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     15     17      1     0     0   1.785D-05   9.355D+00
  F =   9.3547690633130305     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


In [24]:
print("\nModel Summary:")
print(fitted_model.summary())

# 精度結果の表示
print(f'RMSE : {rmse:.2f},  MSE : {mse:.2f},  MAE : {mae:.2f},  MAPE : {mape:.2f}')


Model Summary:
                                     SARIMAX Results                                     
Dep. Variable:                          original   No. Observations:                   73
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood                -682.898
Date:                           Fri, 15 Dec 2023   AIC                           1375.796
Time:                                   18:44:50   BIC                           1386.668
Sample:                               01-01-2022   HQIC                          1380.086
                                    - 03-14-2022                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1020      0.393     -0.259      0.795      -0.872       0.668
ma.L1         -0.3569      

### 残差に正規性が確認できないのでパラメータのグリッドサーチ

In [9]:
# パラメータの範囲を設定
p = d = q = P = D = Q = range(0, 3)  # 0から2までの範囲
s = 7  # 季節性の周期（例：週次データの場合は7）

# ARIMAと季節性パラメータの組み合わせ
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(P, D, Q))]
all_combinations = list(itertools.product(pdq, seasonal_pdq))

best_aic = np.inf
best_params = None
best_model = None

# グリッドサーチ
for param in pdq:
    for seasonal_param in seasonal_pdq:
        try:
            model = SARIMAX(train_df['original'],
                            exog=None, #外生変数なし
                            order=param,
                            seasonal_order=seasonal_param,
                            kalman_filter='exact')

            results = model.fit()

            # AICでモデルを評価
            if results.aic < best_aic:
                best_aic = results.aic
                best_params = (param, seasonal_param)
                best_model = results            

        except:
            continue

# 最適なモデルで予測
pred = best_model.get_forecast(steps=len(test_df))
y_pred = pred.predicted_mean
y_test = test_df['original']

# 評価指標の計算
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 結果の表示
print(f'Best SARIMAX parameters: {best_params}')
print(f'RMSE: {rmse}, MSE: {mse}, MAE: {mae}, MAPE: {mape}')

# 赤池情報量基準（AIC）
print("AIC:", best_aic)

# モデルのサマリーを表示
print("Model Summary:")
print(best_model.summary())

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.24718D+01    |proj g|=  3.55271D-10

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      0      1      0     0     0   3.553D-10   1.247D+01
  F =   12.471759985933176     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.2

/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['kalman_filter']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
 This problem is unconstrained.
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(da


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      8     11      1     0     0   1.236D-05   1.215D+01
  F =   12.150487216764626     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.01360D+01    |proj g|=  5.65183D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates

/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['kalman_filter']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
 This problem is unconstrained.
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(da

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.06232D+00    |proj g|=  2.21986D-01

At iterate    5    f=  9.02895D+00    |proj g|=  1.10778D-02

At iterate   10    f=  9.02847D+00    |proj g|=  1.70202D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     10     12      1     0     0   1.702D-05   9.028D+00
  F =   9.0284697773163796     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


...

/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['kalman_filter']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters 


At iterate    5    f=  8.54194D+00    |proj g|=  3.73632D-03

At iterate   10    f=  8.54187D+00    |proj g|=  3.09601D-04

At iterate   15    f=  8.54170D+00    |proj g|=  8.62344D-03

At iterate   20    f=  8.54019D+00    |proj g|=  2.89160D-03

At iterate   25    f=  8.53986D+00    |proj g|=  2.06876D-03

At iterate   30    f=  8.53978D+00    |proj g|=  1.71670D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     32     34      1     0     0   2.819D-05   8.540D+00
  F =   8.5397783250900865     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machin

/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['kalman_filter']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters 


At iterate   10    f=  8.54294D+00    |proj g|=  1.22162D-02

At iterate   15    f=  8.54174D+00    |proj g|=  5.86523D-04

At iterate   20    f=  8.54171D+00    |proj g|=  2.55977D-03

At iterate   25    f=  8.54058D+00    |proj g|=  2.58625D-03

At iterate   30    f=  8.53988D+00    |proj g|=  1.27099D-03

At iterate   35    f=  8.53976D+00    |proj g|=  5.49427D-05

At iterate   40    f=  8.53976D+00    |proj g|=  2.81911D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     40     42      1     0     0   2.819D-05   8.540D+00
  F =   8.5397641039479240     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH   

/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/statespace/mlemodel.py:3015: RuntimeWarning: divide by zero encountered in divide
  return self.params / self.bse
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:1363: RuntimeWarning: invalid value encountered in divide
  test_statistic = numer_squared_sum / denom_squared_sum
/Users/groov/anaconda3/envs/v_py39/lib/python3.9/site-packages/statsmodels/tsa/stattools.py:691: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


### クロスバリデーション

In [10]:
warnings.simplefilter('ignore')

# パラメータの範囲を設定
p = d = q = P = D = Q = range(0, 3)  # 0から2までの範囲
s = 7  # 季節性の周期（例：週次データの場合は7）

# ARIMAと季節性パラメータの組み合わせ
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(P, D, Q))]
all_combinations = list(itertools.product(pdq, seasonal_pdq))

# グリッドサーチのための準備
tscv = TimeSeriesSplit(n_splits=5)
best_rmse = np.inf
best_aic = np.inf
best_params_rmse = None
best_params_aic = None
best_model_rmse = None
best_model_aic = None

for param in all_combinations:
    rmse_list = []
    aic_list = []
    for train_index, test_index in tscv.split(train_df):
        cv_train, cv_test = train_df.iloc[train_index], train_df.iloc[test_index]

        try:
            model = SARIMAX(cv_train['original'],
                            order=param[0],
                            seasonal_order=param[1],
                            kalman_filter='exact')
            results = model.fit(disp=False,verbose=0)
            predictions = results.forecast(steps=len(cv_test))
            
            # NaN または無限大の値が含まれていないかチェック
            if not np.any(np.isnan(predictions)) and not np.isinf(predictions).any():
                rmse = np.sqrt(mean_squared_error(cv_test['original'], predictions))
                rmse_list.append(rmse)
                aic_list.append(results.aic)
        except:
            continue  # エラーが発生した場合は次の組み合わせに移る

    # RMSEとAICの平均値を計算
    if rmse_list and aic_list:
        avg_rmse = np.mean(rmse_list)
        avg_aic = np.mean(aic_list)
        if avg_rmse < best_rmse:
            best_rmse = avg_rmse
            best_params_rmse = param
            best_model_rmse = results
        if avg_aic < best_aic:
            best_aic = avg_aic
            best_params_aic = param
            best_model_aic = results

# ベストモデルのパラメータとサマリーを表示
print(f'Best SARIMAX parameters based on RMSE: {best_params_rmse}')
print(f'Best SARIMAX parameters based on AIC: {best_params_aic}')
print("\nModel Summary (Best RMSE):")
print(best_model_rmse.summary())
print("\nModel Summary (Best AIC):")
print(best_model_aic.summary())

Best SARIMAX parameters based on RMSE: ((1, 2, 1), (1, 0, 1, 7))
Best SARIMAX parameters based on AIC: ((2, 2, 2), (2, 2, 0, 7))

Model Summary (Best RMSE):
                                     SARIMAX Results                                     
Dep. Variable:                          original   No. Observations:                   61
Model:             SARIMAX(1, 2, 1)x(1, 0, 1, 7)   Log Likelihood                -637.298
Date:                           Fri, 15 Dec 2023   AIC                           1284.595
Time:                                   18:13:54   BIC                           1294.983
Sample:                               01-01-2022   HQIC                          1288.650
                                    - 03-02-2022                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------

In [11]:
print(f'RMSE : {rmse:.2f},  MSE : {mse:.2f},  MAE : {mae:.2f},  MAPE : {mape:.2f}')

RMSE : 10069.15,  MSE : 1293488622632.62,  MAE : 1129273.50,  MAPE : 2785.20
